In [ ]:
pip install dronekit-python pygame ultralytics opencv-python picamera


In [ ]:
python disaster_mission.py


In [ ]:
import time
import csv
import pygame
import cv2
from picamera import PiCamera
from picamera.array import PiRGBArray
from dronekit import connect, VehicleMode, LocationGlobalRelative
import yolov8  # Requires YOLOv8 ONNX/PyTorch installation


In [ ]:
# Configuration
CONNECTION_STRING = '/dev/ttyAMA0'
SAFE_ALTITUDE = 15  # meters
WAYPOINTS = [
    LocationGlobalRelative(-35.363244, 149.168801, SAFE_ALTITUDE),
    LocationGlobalRelative(-35.363300, 149.168900, SAFE_ALTITUDE)
]
BATTERY_WARN = 30  # Percentage
BATTERY_CRITICAL = 15

In [ ]:
# Initialize components
print("Initializing systems...")
vehicle = connect(CONNECTION_STRING, wait_ready=True)
camera = PiCamera()
camera.resolution = (640, 480)
raw_capture = PiRGBArray(camera, size=(640, 480))
model = yolov8.load('yolov8n_survivor.onnx')
pygame.mixer.init()
alert_sound = pygame.mixer.Sound('alert.wav')

In [ ]:
# Create log files
with open('mission_log.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['timestamp', 'lat', 'lon', 'alt', 'detections', 'battery'])

def arm_and_takeoff(target_altitude):
    print("Arming motors")
    vehicle.mode = VehicleMode("GUIDED")
    vehicle.armed = True

    while not vehicle.armed:
        time.sleep(1)

    print("Taking off")
    vehicle.simple_takeoff(target_altitude)

    while True:
        if vehicle.location.global_relative_frame.alt >= target_altitude*0.95:
            print("Reached target altitude")
            break
        time.sleep(1)

def capture_frame():
    raw_capture.truncate(0)
    camera.capture(raw_capture, format='bgr', use_video_port=True)
    return raw_capture.array

def detect_survivors(frame):
    results = model(frame)
    return [obj for obj in results if obj['label'] == 'person']

def log_detection(detections):
    with open('mission_log.csv', 'a') as f:
        writer = csv.writer(f)
        for obj in detections:
            pos = vehicle.location.global_relative_frame
            writer.writerow([
                time.time(),
                pos.lat,
                pos.lon,
                pos.alt,
                obj['confidence'],
                vehicle.battery.level
            ])

def check_failsafe():
    if vehicle.battery.level < BATTERY_CRITICAL:
        print("Critical battery! Landing immediately")
        vehicle.mode = VehicleMode("LAND")
    elif vehicle.battery.level < BATTERY_WARN:
        print("Low battery! Returning to launch")
        vehicle.mode = VehicleMode("RTL")
    
    if vehicle.gps_0.fix_type < 3:
        print("GPS signal lost! Emergency landing")
        vehicle.mode = VehicleMode("LAND")

try:
    arm_and_takeoff(SAFE_ALTITUDE)
    
    for wp in WAYPOINTS:
        print(f"Heading to waypoint {wp}")
        vehicle.simple_goto(wp)
        
        while vehicle.groundspeed > 0.5:  # Wait until reaching waypoint
            check_failsafe()
            time.sleep(1)
        
        frame = capture_frame()
        detections = detect_survivors(frame)
        
        if detections:
            alert_sound.play()
            timestamp = str(time.time()).replace('.','')
            cv2.imwrite(f'detection_{timestamp}.jpg', frame)
            log_detection(detections)
            
        check_failsafe()

finally:
    print("Returning to launch")
    vehicle.mode = VehicleMode("RTL")
    
    while vehicle.armed:
        time.sleep(1)
    
    camera.close()
    pygame.quit()
    vehicle.close()